In [15]:
import time
import sys
import winsound

from selenium import webdriver

from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from get_brand_list import get_brand_list
from get_clothes_info import get_clothes_info
from save_json import save_json
from load_json import load_json
from transformer import transform_datatype_and_soldout

In [19]:
# 1페이지에 존재하는 품목들을 가져오는 함수
def get_product_list():
    css = '#contents > div > div > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    product_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    products = product_list.find_elements_by_xpath(xpath) # 여기에 모든 element가 담김(리스트로)
    return products

# 품목 카테고리 가져오는 함수
def get_product_category():
    """ex: 니트 · 가디건"""

    css = '#contents > div > div > div.xans-element-.xans-product.xans-product-headcategory.title > h2'
    product_category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_category = product_category.text
    print(product_category)
    return product_category

# 품목을 클릭하는 함수
def click_product(iter):
    css1 = '#contents > div > div > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    css2 = 'div > a'
    # 드라이버 재정의
    product_list = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css1)))
    products = product_list.find_elements_by_xpath(xpath)
    # url
    product_link = products[iter].find_element_by_css_selector(css2).get_attribute('href')
    # 박스 클릭
    product_box = WebDriverWait(products[iter], 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css2)))
    product_box.click()
    print(f'link:{product_link}')
    time.sleep(1)
    return product_link

# 품목 이름을 가져오는 함수
def get_product_name():
    class_name = 'ndc_name'
    product_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    product_name = product_name.text
    print(f'name:{product_name}'#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(1) > td > span > br:nth-child(2))
    return product_name

# 품목 브랜드를 가져오는 함수
def get_product_brand():
    """
    명시된 브랜드명을 그대로 가져옴
    우리 브랜드명에 맞춘 변환은 get_brand_list.py 함수에서 함
    실행은 get_clothes_info() 함수에서 받아서 변환
    """
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(2) > td'
    product_brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_brand = product_brand.text
    print(f'brand:{product_brand}')
    return product_brand

# 품목 가격을 가져오는 함수
def get_product_price():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(5) > td > span'
    product_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_price = product_price.text
    product_price = product_price.replace(",", '').replace(" ", "").replace("KRW", '')
    print(f'price:{product_price}')
    return product_price

# 품목 사이즈를 가져오는 함수
def get_product_size():
    id = 'prdDetail'
    product_sizes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, id))).get_attribute("innerHTML")
    product_sizes = product_sizes.strip().split('\n')
    try: # 가슴 사이즈가 있으면 가슴 사이즈만 추출
        product_size = [x for x in product_sizes if '가슴' in x][0].split('가슴')[1].split('cm')[0].strip()
        type_ = '가슴'
    except: # 가슴 사이즈 없고 허리 사이즈 있으면 허리 사이즈 추출
        product_size = [x for x in product_sizes if '허리' in x][0].split('허리')[1].split('cm')[0].strip()
        type_ = '허리'
    print(f'type: {type_}, size:{product_size}')
    return type_, product_size

# 썸네일 이미지를 가져오는 함수
def get_product_thumbnail():
    xpath = '//*[@id="contents"]/div[1]/div[1]/div[2]/div/img'
    product_thumbnail = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).get_attribute('src')
    print(f'thumbnail:{product_thumbnail}')
    return product_thumbnail

# 성별 구분하는 함수
def get_product_sex():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(3) > td'
    product_size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_size = product_size.text
    if product_size[0] == 'W': # 사이즈에 WL, WM 이런 식으로 되어 있으면 여자꺼
        sex = 'women'
    else: # M, L, 이런식이면 남자꺼
        sex = 'men'
    print(f'product size:{product_size}, {sex}')
    return sex
    
# 품목 카테고리 구분하는 함수
def get_product_category_type(product_category, sizing_name):
    if product_category == '반팔 · 반바지':
        if sizing_name == '가슴':
            category = 'top'
        elif sizing_name == '허리':
            category = 'bottom'
    elif product_category == '치마 · 원피스':
        if sizing_name == '가슴':
            category = 'dress'
        elif sizing_name == '허리':
            category = 'skirt'
    elif product_category in ['후디 · 맨투맨', '셔츠 · 긴팔티', '니트 · 가디건']:
        category = 'top'
    elif product_category in ['코트 · 아우터', '점퍼 · 스포츠', '자켓 · 블루종']:
        category = 'outer'
    else: # 가끔 품목에 기타 카테고리가 있음
        category = 'others'
    print(f'category:{category}')
    return category

# 품절 여부 확인하는 함수
def is_product_sold_out():
    """sold out 아이콘이 있는지 체킹해서 품절 여부를 판단한다."""
    
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.headingArea > span.icon > img'
    try:
        product_sold_out = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        product_sold_out.get_attribute('src')
        print(f'품절 상품') # img src가 있으면 sold out 아이콘이 있는 거니까 품절
        return True
    except: # img src가 없으면 판매중
        print(f'판매중인 상품')
        return False

In [26]:
# 바꿨는지 확인
check = input('바꾸셨나요?(y/n)')
if check == 'y':
    pass
else: # 종료
    sys.exit()
driver = webdriver.Chrome(executable_path='chromedriver')
store_name = '릴리즘'
file_name = f'{store_name}.json' ### 바꿔 ###
 
for i in range(13, 15): ##### 바꿔 #####
    URL = f'https://relizm.com/product/menlist.html?cate_no=281&page={i}' ##### 바꿔 #####
    driver.get(url=URL)
    total_list = load_json(file_name)
    product_list = get_product_list()
    for iter in range(len(product_list)):
        print('----------------{} page, {} -----------------'.format(i, iter+1))
        # print(product_list[i].get_attribute('innerHTML'))

        product_categories = get_product_category()
        product_link = click_product(iter)
        try:
            product_name = get_product_name()
            product_brand_before = get_product_brand()
            product_price = get_product_price()
            product_sale_price = get_product_sale_price()
            product_type, product_size = get_product_size()
            product_thumbnail = get_product_thumbnail()
            product_gender = get_product_sex()
            product_category = get_product_category_type(product_categories)
            is_sold_out = is_product_sold_out()
            product_brand_after = get_brand_list(product_brand_before)
        except:
            time.sleep(0.5)
            driver.get(url=URL)
            continue
        json_ = get_clothes_info(store_name, product_sale_price, product_name, product_category, product_gender,
 product_brand_after, product_price, product_thumbnail, product_link, is_sold_out, product_size, product_type)
        total_list.append(json_)
        time.sleep(0.5)
        driver.get(url=URL)
    save_json(data=total_list, filename=file_name)

winsound.Beep(440, 500)

In [ ]:
save_json(data=total_list, filename=file_name)

In [ ]:
original, onlysale = transform_datatype_and_soldout(file_name)